## Limpieza de Datos: 
## Encuestas de Patrones Infonavit

Carga de librerías:

In [251]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder,KBinsDiscretizer
from sklearn.impute import SimpleImputer
import sys, os
import re

In [252]:
os.getcwd()

'/home/mariana/Documents/TESIS_MS'

In [253]:
%reload_ext autoreload
%autoreload 
from utils.funcs_utils import *

Lectura de datos:

In [254]:
df= pd.read_csv("/home/mariana/Documents/TESIS_MS/BaseAgregada.csv")
df.rename(columns={col: col.lower() for col in df.columns.values},inplace=True)

/home/mariana/.pyenv/versions/deep_learning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (5,21,23,24,26,27,35,37,67,68,69,70,71,74,75,76,77,78,79,80,83,84,85,86,87,88,89,90,92,93,95,96,97,99,100,101,102,105,106,107,108,109,110,124,132,133,143,144,145,146,147,148,149,151,152,153,167,168,169,170,174,176,178,189,190,192,193,195,196,205,210,211) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Se cambia el nombre de las columnas a minúscula:

In [255]:
count_unique_obs(df)

history_id_crm       7834
estrato_nuevo           3
estrato_nuevo_txt       3
ponderador_sgpff       54
pond_relativizado      51
region                 19
region_txt             10
empresas                2
p1_1_a                 14
p1_1_b                 94
p1_2                    8
p1_2_6_tx             344
p1_2_6_cod              8
p1_3_tx              6008
p1_3_cod               22
p1_4                   10
p1_4_9_tx             772
p1_4_9_cod             14
p1_5                    3
p2_1_tx1             3112
p2_1_cod11             20
p2_1_cod12              8
p2_1_tx2             2900
p2_1_1cod21            40
p2_1_1cod22             6
p2_1_tx3             1820
p2_1_2cod31            40
p2_1_2cod32             7
p2_2                  512
p2_3                   75
p2_4                  132
p2_5_a                113
p2_5_b                 83
p2_6                  505
p2_7                    4
p2_8                    9
p2_9                    4
p2_10                   7
p2_11       

In [256]:
df.dtypes

history_id_crm         int64
estrato_nuevo          int64
estrato_nuevo_txt     object
ponderador_sgpff     float64
pond_relativizado    float64
region                object
region_txt            object
empresas               int64
p1_1_a                 int64
p1_1_b                 int64
p1_2                   int64
p1_2_6_tx             object
p1_2_6_cod           float64
p1_3_tx               object
p1_3_cod              object
p1_4                   int64
p1_4_9_tx             object
p1_4_9_cod           float64
p1_5                   int64
p2_1_tx1              object
p2_1_cod11             int64
p2_1_cod12            object
p2_1_tx2              object
p2_1_1cod21           object
p2_1_1cod22           object
p2_1_tx3              object
p2_1_2cod31           object
p2_1_2cod32           object
p2_2                   int64
p2_3                   int64
p2_4                   int64
p2_5_a                 int64
p2_5_b                 int64
p2_6                   int64
p2_7          

Contamos el total de "sí=1" para ver en cuánto recurso invirtió

In [257]:
is_1= lambda x:int(x==1)

In [258]:
count_19 = lambda row: is_1(row['p2_19_a']) + is_1(row['p2_19_b'])+is_1(row['p2_19_c']) + is_1(row['p2_19_d'])+ is_1(row['p2_19_e']) + is_1(row['p2_19_f'])+ is_1(row['p2_19_g']) + is_1(row['p2_19_h'])+ is_1(row['p2_19_i']) + is_1(row['p2_19_j'])+is_1(row['p2_19_k']) + is_1(row['p2_19_l'])+ is_1(row['p2_19_m']) + is_1(row['p2_19_n'])+is_1(row['p2_19_o']) 
df['p2_19'] = df.apply(count_19 ,axis=1)

In [259]:
count_20 = lambda row: is_1(row['p2_20_a']) + is_1(row['p2_20_b'])+is_1(row['p2_20_c']) + is_1(row['p2_20_d']) 
df['p2_20'] = df.apply(count_20 ,axis=1)

In [260]:
count_19_20 = lambda row: is_1(row['p2_19_a']) + is_1(row['p2_19_b'])+is_1(row['p2_19_c']) + is_1(row['p2_19_d'])+ is_1(row['p2_19_e']) + is_1(row['p2_19_f'])+ is_1(row['p2_19_g']) + is_1(row['p2_19_h'])+ is_1(row['p2_19_i']) + is_1(row['p2_19_j'])+is_1(row['p2_19_k']) + is_1(row['p2_19_l'])+ is_1(row['p2_19_m']) + is_1(row['p2_19_n'])+is_1(row['p2_19_n']) + is_1(row['p2_20_a']) + is_1(row['p2_20_b'])+is_1(row['p2_20_c']) + is_1(row['p2_20_d'])
df['inversion_total'] = df.apply(count_19_20 ,axis=1)

Se suman únicamente los créditos autorizados.

In [261]:
df['2_22_autorizados'] = df['p2_22_a'] + df['p2_22_b'] 

Solicitud de créditos en el trimestre anterior.

In [262]:
df["p2_27_a"] = pd.to_numeric(df.p2_27_a, errors='coerce')
df["p2_27_b"] = pd.to_numeric(df.p2_27_b, errors='coerce')
df["p2_27_c"] = pd.to_numeric(df.p2_27_c, errors='coerce')

In [263]:
count_27 = lambda row: is_1(row['p2_27_a']) + is_1(row['p2_27_b'])+is_1(row['p2_27_c'])
df['p2_27_bis'] = df.apply(count_27,axis=1)

La variables 2_27 establece si en algún mes del trimestre anterior solicitó un crédito es 1, 0 en otro caso.

In [264]:
df['2_27'] = np.where(df['p2_27_bis']>0,1,0)

Si pagó impuestos y/o contribuciones el trimestre anterior: 1=Sí, 2=No.

In [265]:
is_2= lambda x:int(x==2)
is_98= lambda x:int(x==98)
is_99= lambda x:int(x==99)

In [266]:
count_imp = lambda row: is_1(row['p2_41_a']) + is_1(row['p2_41_b'])+is_1(row['p2_41_c'])
df['p2_41_imp'] = df.apply(count_imp ,axis=1)

In [267]:
count_contr= lambda row: is_1(row['p2_41_d']) + is_1(row['p2_41_e'])+is_1(row['p2_41_f'])
df['p2_41_contr'] = df.apply(count_contr ,axis=1)

In [268]:
count_none= lambda row: is_2(row['p2_41_a']) + is_2(row['p2_41_b'])+is_2(row['p2_41_c'])+ is_2(row['p2_41_d']) + is_2(row['p2_41_e'])+is_2(row['p2_41_f'])+ is_2(row['p2_41_g'])
df['p2_41_none'] = df.apply(count_none ,axis=1)

In [269]:
count_imp2 = lambda row: is_2(row['p2_41_a']) + is_2(row['p2_41_b'])+is_2(row['p2_41_c'])
df['p2_41_imp2'] = df.apply(count_imp2 ,axis=1)

In [270]:
count_contr2= lambda row: is_2(row['p2_41_d']) + is_2(row['p2_41_e'])+is_2(row['p2_41_f'])
df['p2_41_contr2'] = df.apply(count_contr2,axis=1)

In [271]:
count_98c= lambda row: is_98(row['p2_41_d']) + is_98(row['p2_41_e'])+is_98(row['p2_41_f'])
df['p2_41_contr98'] = df.apply(count_98c,axis=1)

In [272]:
count_98i= lambda row: is_98(row['p2_41_a']) + is_98(row['p2_41_b'])+is_98(row['p2_41_c'])
df['p2_41_imp98'] = df.apply(count_98i,axis=1)

In [273]:
count_99c= lambda row: is_99(row['p2_41_d']) + is_99(row['p2_41_e'])+is_99(row['p2_41_f'])
df['p2_41_contr99'] = df.apply(count_99c,axis=1)

In [274]:
count_99i= lambda row: is_99(row['p2_41_a']) + is_99(row['p2_41_b'])+is_99(row['p2_41_c'])
df['p2_41_imp99'] = df.apply(count_99i,axis=1)

In [275]:
count_both= lambda row: is_1(row['p2_41_a']) + is_1(row['p2_41_b'])+is_1(row['p2_41_c'])+ is_1(row['p2_41_d']) + is_1(row['p2_41_e'])+is_1(row['p2_41_f'])+ is_1(row['p2_41_g'])
df['p2_41_both'] = df.apply(count_both ,axis=1)

In [276]:
count_g= lambda row: is_1(row['p2_41_g'])
df['2_41_g'] = df.apply(count_g,axis=1)

Si pagó a tiempo los impuestos o contribuciones el trimetre anterior: 1=Sí, 2=No.

In [277]:
count_imp42 = lambda row: is_2(row['p2_42_a']) + is_2(row['p2_42_b'])+is_2(row['p2_42_c'])
df['p2_42_imp'] = df.apply(count_imp42 ,axis=1)

In [278]:
count_contr42= lambda row: is_2(row['p2_42_d']) + is_2(row['p2_42_e'])+is_2(row['p2_42_f'])
df['p2_42_contr'] = df.apply(count_contr42 ,axis=1)

In [279]:
count_all= lambda row: is_2(row['p2_42_a']) + is_2(row['p2_42_b'])+is_2(row['p2_42_c'])+ is_2(row['p2_42_d']) + is_2(row['p2_42_e'])+is_2(row['p2_42_f'])+ is_2(row['p2_42_g'])
df['p2_42_all'] = df.apply(count_all ,axis=1)

In [280]:
count_none42= lambda row: is_1(row['p2_42_a']) + is_1(row['p2_42_b'])+is_1(row['p2_42_c'])+ is_1(row['p2_42_d']) + is_1(row['p2_42_e'])+is_1(row['p2_42_f'])+ is_1(row['p2_42_g'])
df['p2_42_none'] = df.apply(count_none42 ,axis=1)

In [281]:
count_98c= lambda row: is_98(row['p2_42_d']) + is_98(row['p2_42_e'])+is_98(row['p2_42_f'])
df['p2_42_contr98'] = df.apply(count_98c,axis=1)

In [282]:
count_98i= lambda row: is_98(row['p2_42_a']) + is_98(row['p2_42_b'])+is_98(row['p2_42_c'])
df['p2_42_imp98'] = df.apply(count_98i,axis=1)

In [283]:
count_99c= lambda row: is_99(row['p2_42_d']) + is_99(row['p2_42_e'])+is_99(row['p2_42_f'])
df['p2_42_contr99'] = df.apply(count_99c,axis=1)

In [284]:
count_99i= lambda row: is_99(row['p2_42_a']) + is_99(row['p2_42_b'])+is_99(row['p2_42_c'])
df['p2_42_imp99'] = df.apply(count_99i,axis=1)

In [285]:
count_g= lambda row: is_2(row['p2_42_g'])
df['2_42_g2'] = df.apply(count_g,axis=1)

Dificultades para pagar durante este trimestre impuestos y/o contribuciones: 1=Sí, 2=No

In [286]:
count_imp = lambda row: is_1(row['p2_44_a']) + is_1(row['p2_44_b'])+is_1(row['p2_44_c'])
df['p2_44_imp'] = df.apply(count_imp ,axis=1)

In [287]:
count_contr= lambda row: is_1(row['p2_44_d']) + is_1(row['p2_44_e'])+is_1(row['p2_44_f'])
df['p2_44_contr'] = df.apply(count_contr ,axis=1)

In [288]:
count_none= lambda row: is_2(row['p2_44_a']) + is_2(row['p2_44_b'])+is_2(row['p2_44_c'])+ is_2(row['p2_44_d']) + is_2(row['p2_44_e'])+is_2(row['p2_44_f'])
df['p2_44_none'] = df.apply(count_none ,axis=1)

In [289]:
count_both= lambda row: is_1(row['p2_44_a']) + is_1(row['p2_44_b'])+is_1(row['p2_44_c'])+ is_1(row['p2_44_d']) + is_1(row['p2_44_e'])+is_1(row['p2_44_f'])
df['p2_44_both'] = df.apply(count_both ,axis=1)

In [290]:
count_98c= lambda row: is_98(row['p2_44_d']) + is_98(row['p2_44_e'])+is_98(row['p2_44_f'])
df['p2_44_contr98'] = df.apply(count_98c,axis=1)

In [291]:
count_98i= lambda row: is_98(row['p2_44_a']) + is_98(row['p2_44_b'])+is_98(row['p2_44_c'])
df['p2_44_imp98'] = df.apply(count_98i,axis=1)

In [292]:
count_99i= lambda row: is_99(row['p2_44_a']) + is_99(row['p2_44_b'])+is_99(row['p2_44_c'])
df['p2_44_imp99'] = df.apply(count_99i,axis=1)

In [293]:
count_99c= lambda row: is_99(row['p2_44_d']) + is_99(row['p2_44_e'])+is_99(row['p2_44_f'])
df['p2_44_contr99'] = df.apply(count_99c,axis=1)

Dificultades para pagar impuestos y/o contribuciones lo que resta del año: 1=Sí, 2=No

In [294]:
count_imp = lambda row: is_1(row['p2_45_a']) + is_1(row['p2_45_b'])+is_1(row['p2_45_c'])
df['p2_45_imp'] = df.apply(count_imp ,axis=1)

In [295]:
count_contr= lambda row: is_1(row['p2_45_d']) + is_1(row['p2_45_e'])+is_1(row['p2_45_f'])
df['p2_45_contr'] = df.apply(count_contr ,axis=1)

In [296]:
count_none= lambda row: is_2(row['p2_45_a']) + is_2(row['p2_45_b'])+is_2(row['p2_45_c'])+ is_2(row['p2_45_d']) + is_2(row['p2_45_e'])+is_2(row['p2_45_f'])
df['p2_45_none'] = df.apply(count_none ,axis=1)

In [297]:
count_both= lambda row: is_1(row['p2_45_a']) + is_1(row['p2_45_b'])+is_1(row['p2_45_c'])+ is_1(row['p2_45_d']) + is_1(row['p2_45_e'])+is_1(row['p2_45_f'])
df['p2_45_both'] = df.apply(count_both ,axis=1)

In [298]:
count_98c= lambda row: is_98(row['p2_45_d']) + is_98(row['p2_45_e'])+is_98(row['p2_45_f'])
df['p2_45_contr98'] = df.apply(count_98c,axis=1)

In [338]:
count_99c= lambda row: is_99(row['p2_45_d']) + is_99(row['p2_45_e'])+is_99(row['p2_45_f'])
df['p2_45_contr99'] = df.apply(count_99c,axis=1)

In [299]:
count_98i= lambda row: is_98(row['p2_45_a']) + is_98(row['p2_45_b'])+is_98(row['p2_45_c'])
df['p2_45_imp98'] = df.apply(count_98i,axis=1)

In [300]:
count_99i= lambda row: is_99(row['p2_45_a']) + is_99(row['p2_45_b'])+is_99(row['p2_45_c'])
df['p2_45_imp99'] = df.apply(count_99i,axis=1)

Convertimos todas las variables catagóricas que están como float a numéricas para homologar categorías.

In [339]:
pd.set_option('display.max_rows', 10)
convert_to_num(df,vars_to_cat)

,history_id_crm,estrato_nuevo,estrato_nuevo_txt,ponderador_sgpff,pond_relativizado,...,p2_45_both,p2_45_contr98,p2_45_imp98,p2_45_imp99,p2_45_contr99
0,7521766,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,6,0,0,0,0
1,7544090,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,0,0,0,0,0
2,7543953,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,0,0,0,0,0
3,7536178,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,0,0,0,0,0
4,7521922,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7829,224423829,3,NaN,NaN,0.13,...,0,0,0,0,0
7830,224637558,1,NaN,NaN,4.56,...,0,0,0,0,0
7831,224475564,2,NaN,NaN,0.15,...,0,0,0,0,0
7832,223989494,3,NaN,NaN,0.05,...,0,0,0,0,0


Algunas no las hace la función, por lo que se ejecutan manualmente. 

In [340]:
df['p2_16'] = df['p2_16'].fillna(0).astype(int)
df['p2_27'] = df['p2_27'].fillna(0).astype(int)
df['p2_34_a'] = df['p2_34_a'].fillna(0).astype(int)
df['p2_34_b'] = df['p2_34_b'].fillna(0).astype(int)
df['p2_34_c'] = df['p2_34_c'].fillna(0).astype(int)
df['p2_34_d'] = df['p2_34_d'].fillna(0).astype(int)
df['p2_34_e'] = df['p2_34_e'].fillna(0).astype(int)
df['p3_12'] = df['p3_12'].fillna(0).astype(int)
df['p3_13'] = df['p3_13'].fillna(0).astype(int)
df['p5_1'] = df['p5_1'].fillna(0).astype(int)
df['p5_2'] = df['p5_2'].fillna(0).astype(int)
df['p5_4'] = df['p5_4'].fillna(0).astype(int)
df['p5_5'] = df['p5_5'].fillna(0).astype(int)


In [341]:
pd.set_option('display.max_rows', None)
df.dtypes

history_id_crm         int64
estrato_nuevo          int64
estrato_nuevo_txt     object
ponderador_sgpff     float64
pond_relativizado    float64
region                object
region_txt            object
empresas               int64
p1_1_a                 int64
p1_1_b                 int64
p1_2                   int64
p1_2_6_tx             object
p1_2_6_cod           float64
p1_3_tx               object
p1_3_cod              object
p1_4                   int64
p1_4_9_tx             object
p1_4_9_cod           float64
p1_5                   int64
p2_1_tx1              object
p2_1_cod11             int64
p2_1_cod12            object
p2_1_tx2              object
p2_1_1cod21           object
p2_1_1cod22           object
p2_1_tx3              object
p2_1_2cod31           object
p2_1_2cod32           object
p2_2                   int64
p2_3                   int64
p2_4                   int64
p2_5_a                 int64
p2_5_b                 int64
p2_6                   int64
p2_7          

Convertimos todas las variables en catagóricas.

In [342]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)
convert_object(df, vars_to_cat)

,history_id_crm,estrato_nuevo,estrato_nuevo_txt,ponderador_sgpff,pond_relativizado,...,p2_45_both,p2_45_contr98,p2_45_imp98,p2_45_imp99,p2_45_contr99
0,7521766,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,6,0,0,0,0
1,7544090,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,0,0,0,0,0
2,7543953,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,0,0,0,0,0
3,7536178,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,0,0,0,0,0
4,7521922,1,Pequeña y micro (1 a 50 empleados),72.128114,NaN,...,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7829,224423829,3,NaN,NaN,0.13,...,0,0,0,0,0
7830,224637558,1,NaN,NaN,4.56,...,0,0,0,0,0
7831,224475564,2,NaN,NaN,0.15,...,0,0,0,0,0
7832,223989494,3,NaN,NaN,0.05,...,0,0,0,0,0


In [343]:
pd.set_option('display.max_rows', None)
df.dtypes

history_id_crm         int64
estrato_nuevo          int64
estrato_nuevo_txt     object
ponderador_sgpff     float64
pond_relativizado    float64
region                object
region_txt            object
empresas               int64
p1_1_a                 int64
p1_1_b                 int64
p1_2                  object
p1_2_6_tx             object
p1_2_6_cod           float64
p1_3_tx               object
p1_3_cod              object
p1_4                   int64
p1_4_9_tx             object
p1_4_9_cod           float64
p1_5                   int64
p2_1_tx1              object
p2_1_cod11             int64
p2_1_cod12            object
p2_1_tx2              object
p2_1_1cod21           object
p2_1_1cod22           object
p2_1_tx3              object
p2_1_2cod31           object
p2_1_2cod32           object
p2_2                   int64
p2_3                   int64
p2_4                   int64
p2_5_a                 int64
p2_5_b                 int64
p2_6                   int64
p2_7          

In [344]:
data_profiling_categ(df, vars_to_cat)

*********************************
Variable Categorica p1_2
*********************************


Info,p1_2
Num_Registros,7834
Num_de_categorias,8
Moda,5
Valores_faltantes,0
Top1,"[5, 3079]"
Top2,"[4, 1496]"
Top3,"[2, 1244]"


None
Valores de las categorias y sus proporciones


,Observaciones,proporción
Categoría,,
5,3079,39.3%
4,1496,19.1%
2,1244,15.9%
3,1155,14.7%
6,417,5.3%
1,360,4.6%
99,64,0.8%
98,19,0.2%


None



*********************************
Variable Categorica p1_4
*********************************


...

None



*********************************
Variable Categorica p6_3
*********************************


Info,p6_3
Num_Registros,7834
Num_de_categorias,15
Moda,10
Valores_faltantes,0
Top1,"[10, 3017]"
Top2,"[11, 2974]"
Top3,"[9, 411]"


None
Valores de las categorias y sus proporciones


,Observaciones,proporción
Categoría,,
10,3017,38.5%
11,2974,38.0%
9,411,5.2%
12,400,5.1%
8,379,4.8%
7,212,2.7%
99,191,2.4%
5,97,1.2%
6,58,0.7%


None





In [141]:
data_profiling_categ(df, vars_num)

*********************************
Variable Categorica p1_2
*********************************


Info,p1_2
Num_Registros,7834
Num_de_categorias,8
Moda,5
Valores_faltantes,0
Top1,"[5, 3079]"
Top2,"[4, 1496]"
Top3,"[2, 1244]"


None
Valores de las categorias y sus proporciones


,Observaciones,proporción
Categoría,,
5,3079,39.3%
4,1496,19.1%
2,1244,15.9%
3,1155,14.7%
6,417,5.3%
1,360,4.6%
99,64,0.8%
98,19,0.2%


None



*********************************
Variable Categorica p1_4
*********************************


...

None



*********************************
Variable Categorica p6_3
*********************************


Info,p6_3
Num_Registros,7834
Num_de_categorias,15
Moda,10
Valores_faltantes,0
Top1,"[10, 3017]"
Top2,"[11, 2974]"
Top3,"[9, 411]"


None
Valores de las categorias y sus proporciones


,Observaciones,proporción
Categoría,,
10,3017,38.5%
11,2974,38.0%
9,411,5.2%
12,400,5.1%
8,379,4.8%
7,212,2.7%
99,191,2.4%
5,97,1.2%
6,58,0.7%


None



